In [1]:
import os
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Cassandra
from dotenv import load_dotenv
import cassio

In [2]:
load_dotenv()

True

## Getting API keys and token from environment

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")
ASTRA_DB_TOKEN = os.getenv("ASTRADBTOKEN")
ASTRA_DB_ID = os.getenv("ASTRADBID")


## Connecting to the AstraDB

In [5]:
cassio.init(token=ASTRA_DB_TOKEN, database_id=ASTRA_DB_ID)

## Extracting content from webpage

In [6]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(web_paths=("https://www.ibm.com/think/topics/large-language-models",),
                       bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                        ["h1", "h2", "h3", "p"]
                       )))

text_documents = loader.load()
text_documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.ibm.com/think/topics/large-language-models'}, page_content='\n                                   \n\n\n\n  \n    What are large language models (LLMs)?\n\n\n\n\n\n\n    \n\n\n                               \n        \n\n\n\n  \n    Author\n\n\n\n\n\n\n    \n\n\n    Staff Editor, AI ModelsIBM Think\r\n        What are LLMs?\r\n    Large language models (LLMs) are a category of deep learning models trained on immense amounts of data, making them capable of understanding and generating natural language and other types of content to perform a wide range of tasks. LLMs are built on a type of neural network architecture called a transformer which excels at handling sequences of words and capturing patterns in text. LLMs work as giant statistical prediction machines that repeatedly predict the next word in a sequence. They learn patterns in their text and generate language that follows those patterns.LLMs represent a major leap in how humans interact

## Creating the extracted text into chunks

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [8]:
docs = splitter.split_documents(text_documents)
docs[:5]

[Document(metadata={'source': 'https://www.ibm.com/think/topics/large-language-models'}, page_content='What are large language models (LLMs)?\n\n\n\n\n\n\n    \n\n\n                               \n        \n\n\n\n  \n    Author'),
 Document(metadata={'source': 'https://www.ibm.com/think/topics/large-language-models'}, page_content='Staff Editor, AI ModelsIBM Think\r\n        What are LLMs?'),
 Document(metadata={'source': 'https://www.ibm.com/think/topics/large-language-models'}, page_content='Large language models (LLMs) are a category of deep learning models trained on immense amounts of data, making them capable of understanding and generating natural language and other types of content to perform a wide range of tasks. LLMs are built on a type of neural network architecture called a transformer which excels at handling sequences of words and capturing patterns in text. LLMs work as giant statistical prediction machines that repeatedly predict the next word in a sequence. They lear

## Converting data into vectors and store in ASTRADB

In [9]:

embedd = OllamaEmbeddings(model="gemma:2b", )
astra_vector_store = Cassandra(
    embedding=embedd, table_name='astradb_demo',
    session=None,keyspace=None
    )

astra_vector_store

C:\Users\hp\AppData\Local\Temp\ipykernel_3608\184084063.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embedd = OllamaEmbeddings(model="gemma:2b", )


In [10]:
astra_vector_store.add_documents(docs)
print("Added %i docs" %len(docs))

Added 39 docs


In [11]:
res = astra_vector_store.similarity_search(query="What are LLMs?", k=4)
for docs in res:
    print(docs.page_content)
    print("******")

Staff Editor, AI ModelsIBM Think
        What are LLMs?
******
Staff Editor, AI ModelsIBM Think
        What are LLMs?
******
order to better understand customer feedback at scale. Language translation: Automated translation provides wider coverage to organizations across languages and geographies with fluent translations and multilingual capabilities.Reasoning: LLMs can solve math problems, plan multi-step processes and explain complex concepts in simpler terms.
******
order to better understand customer feedback at scale. Language translation: Automated translation provides wider coverage to organizations across languages and geographies with fluent translations and multilingual capabilities.Reasoning: LLMs can solve math problems, plan multi-step processes and explain complex concepts in simpler terms.
******


## Creating LLM using Groq and Prompt

In [14]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGroq(groq_api_key=groq_api_key,
                model_name="llama-3.3-70b-versatile")


prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on context.
    Think step by step before providing a detailed answer.
    At the end of the response always the user satisfaction.
    <context>
    {context}
    </context>
    
    Question: {question}"""
)


retriever = astra_vector_store.as_retriever(search_kwargs={"k":4})

In [20]:
from langchain_core.runnables import RunnablePassthrough


chain = ({"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        )

chain

{
  context: VectorStoreRetriever(tags=['Cassandra', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.cassandra.Cassandra object at 0x000001387AC76890>, search_kwargs={'k': 4}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n    Answer the questions based on context.\n    Think step by step before providing a detailed answer.\n    At the end of the response always the user satisfaction.\n    <context>\n    {context}\n    </context>\n\n    Question: {question}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001380084C210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001380084A7D0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_

In [21]:
answer = chain.invoke("How do LLMs perform in real world use cases?")
answer.content

"To address the question of how Large Language Models (LLMs) perform in real-world use cases, let's analyze the information provided in the context step by step.\n\n1. **Understanding LLMs**: The documents provided are from the same source, indicating a consistent perspective on LLMs. The initial documents introduce the concept of LLMs but do not delve into their performance in real-world scenarios directly.\n\n2. **Real-World Applications**: The documents with IDs 'a2e5c889c111458facd5fd0ad1db9e89' and 'f972c26b00c8424fa55d589669d785b8' provide insight into the real-world applications of LLMs. They mention three key areas:\n   - **Customer Feedback Analysis**: LLMs are used to better understand customer feedback at scale. This implies that LLMs can process and analyze large volumes of customer feedback, potentially identifying trends, sentiments, and areas for improvement.\n   - **Language Translation**: LLMs enable automated translation, providing organizations with the capability to